In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Graph_Neural_Network')

In [3]:
!pip install torch_geometric==2.5.0
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.1 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import TAGConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

In [5]:
# ====================== 数据加载 ======================
df_classes = pd.read_csv('./data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
df_edges = pd.read_csv('./data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
df_features = pd.read_csv('./data/elliptic_bitcoin_dataset/elliptic_txs_features.csv', header=None)

In [6]:
# 删除 unknown 类别，仅保留合法/非法交易（1/2）
df_classes = df_classes[df_classes['class'] != 'unknown']
df_classes['class'] = df_classes['class'].map({'1': 1, '2': 0})  # 1: illicit, 0: licit

# 合并特征与类别标签
df_merge = df_features.merge(df_classes, how='inner', right_on="txId", left_on=0)
df_merge = df_merge.drop(['txId'], axis=1)

In [7]:
# ====================== 图构建 ======================
nodes = df_merge[0].values
map_id = {j: i for i, j in enumerate(nodes)}

edges = df_edges[df_edges.txId1.isin(map_id) & df_edges.txId2.isin(map_id)].copy()
edges.txId1 = edges.txId1.map(map_id)
edges.txId2 = edges.txId2.map(map_id)
edges = edges.astype(int)
edge_index = np.array(edges.values).T
edge_index = torch.tensor(edge_index, dtype=torch.long).contiguous()
weights = torch.tensor([1] * edge_index.shape[1], dtype=torch.float32)

labels = torch.tensor(df_merge['class'].values, dtype=torch.float32)
node_features = torch.tensor(np.array(df_merge.drop([0, 'class', 1], axis=1).values), dtype=torch.float32)

elliptic_dataset = Data(x=node_features, edge_index=edge_index, edge_weights=weights, y=labels)

In [8]:
# ====================== 超参数与划分 ======================
seed = 0
learning_rate = 0.005
weight_decay = 1e-5
input_dim = 165
output_dim = 1
hidden_size = 150
num_epochs = 400
checkpoints_dir = './result/models/elliptic_tgnn'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# 所有节点都是有标签样本，划分训练/验证/测试集
all_idx = np.arange(len(labels))
y_all = labels[all_idx]

# 按70:15:15的比例划分训练集、验证集、测试集
train_idx, temp_idx = train_test_split(all_idx, test_size=0.3, random_state=seed, stratify=y_all)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=seed, stratify=labels[temp_idx])

elliptic_dataset.train_idx = torch.tensor(train_idx, dtype=torch.long)
elliptic_dataset.val_idx = torch.tensor(val_idx, dtype=torch.long)
elliptic_dataset.test_idx = torch.tensor(test_idx, dtype=torch.long)


In [10]:
# ====================== 模型定义 ======================
class TGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, K=3):
        super().__init__()
        self.norm1 = torch.nn.BatchNorm1d(dim_in)
        self.gat1 = TAGConv(dim_in, dim_h, K)
        self.norm2 = torch.nn.BatchNorm1d(dim_h)
        self.gat2 = TAGConv(dim_h, dim_out, K)

    def forward(self, x, edge_index):
        h = self.norm1(x)
        h = self.gat1(h, edge_index)
        h = self.norm2(h)
        h = F.leaky_relu(h)
        out = self.gat2(h, edge_index)
        return out

def accuracy(y_pred, y_test, prediction_threshold=0.5):
    y_pred_label = (torch.sigmoid(y_pred) > prediction_threshold).float()
    correct_results_sum = (y_pred_label == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    return acc

In [11]:
# ====================== Activated Gradient 优化器 ======================
class ActivatedAdam(torch.optim.Adam):
    """
    Activated Gradients for Deep Neural Networks (ICLR 2021)
    在反向传播后、参数更新前，对梯度进行激活函数式调制：
    g' = tanh(alpha * g)
    """
    def __init__(self, params, lr=0.001, weight_decay=0.0, activation='tanh', alpha=1.0):
        super().__init__(params, lr=lr, weight_decay=weight_decay)
        self.activation = activation
        self.alpha = alpha

    @torch.no_grad()
    def step(self, closure=None):
        # 在执行优化器更新前，对梯度进行激活调制
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                g = p.grad.data
                if self.activation == 'tanh':
                    g_activated = torch.tanh(self.alpha * g)
                elif self.activation == 'sigmoid':
                    g_activated = torch.sigmoid(self.alpha * g) * 2 - 1
                elif self.activation == 'relu':
                    g_activated = F.relu(g)
                else:
                    g_activated = g
                p.grad.data = g_activated
        return super().step(closure)

In [12]:
# ====================== 模型训练函数（使用 Activated Gradient） ======================
def train(model, data, criterion, optimizer, num_epochs, checkpoint_dir, model_filename):
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    best_loss = float('inf')
    model.train()

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        pred = model(data.x, data.edge_index)
        loss = criterion(pred[data.train_idx], data.y[data.train_idx].unsqueeze(1))
        acc = accuracy(pred[data.train_idx], data.y[data.train_idx].unsqueeze(1))

        loss.backward()
        optimizer.step()  # 自动执行 Activated Gradient

        val_loss = criterion(pred[data.val_idx], data.y[data.val_idx].unsqueeze(1))
        val_acc = accuracy(pred[data.val_idx], data.y[data.val_idx].unsqueeze(1))

        if epoch % 10 == 0:
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.4f} | '
                  f'Val Acc: {val_acc*100:.2f}%')
            if val_loss < best_loss:
                best_loss = val_loss
                print('Saving best model state')
                checkpoint = {'state_dict': model.state_dict()}
                torch.save(checkpoint, os.path.join(checkpoint_dir, model_filename))

    return model

# ====================== 测试函数 ======================
def test(model, data):
    model.eval()
    preds = model(data.x, data.edge_index)
    preds = ((torch.sigmoid(preds) > 0.5).float() * 1).squeeze(1)
    return preds

# ====================== 模型训练与评估 ======================
torch.manual_seed(seed)

model = TGNN(input_dim, hidden_size, output_dim, K=3).to(device)
data_train = elliptic_dataset.to(device)

# 使用 ActivatedAdam 替代原始 Adam 优化器
optimizer = ActivatedAdam(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
    activation='tanh',  # 激活函数类型
    alpha=0.8           # 梯度缩放系数
)

criterion = torch.nn.BCEWithLogitsLoss()

# 模型训练
train(model, data_train, criterion, optimizer, num_epochs, checkpoints_dir, 'tgnn_best_model.pth.tar')

Epoch   0 | Train Loss: 0.688 | Train Acc:  47.30% | Val Loss: 0.6869 | Val Acc: 47.97%
Saving best model state
Epoch  10 | Train Loss: 0.192 | Train Acc:  90.86% | Val Loss: 0.1921 | Val Acc: 90.81%
Saving best model state
Epoch  20 | Train Loss: 0.157 | Train Acc:  94.01% | Val Loss: 0.1587 | Val Acc: 94.09%
Saving best model state
Epoch  30 | Train Loss: 0.140 | Train Acc:  96.22% | Val Loss: 0.1431 | Val Acc: 96.16%
Saving best model state
Epoch  40 | Train Loss: 0.124 | Train Acc:  96.68% | Val Loss: 0.1286 | Val Acc: 96.45%
Saving best model state
Epoch  50 | Train Loss: 0.109 | Train Acc:  96.91% | Val Loss: 0.1159 | Val Acc: 96.55%
Saving best model state
Epoch  60 | Train Loss: 0.099 | Train Acc:  97.10% | Val Loss: 0.1071 | Val Acc: 96.78%
Saving best model state
Epoch  70 | Train Loss: 0.091 | Train Acc:  97.29% | Val Loss: 0.1018 | Val Acc: 96.86%
Saving best model state
Epoch  80 | Train Loss: 0.085 | Train Acc:  97.43% | Val Loss: 0.0978 | Val Acc: 96.96%
Saving best mode

TGNN(
  (norm1): BatchNorm1d(165, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gat1): TAGConv(165, 150, K=3)
  (norm2): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gat2): TAGConv(150, 1, K=3)
)

In [13]:
# ====================== 模型测试 ======================
y_test_preds = test(model, data_train)

model.eval()
with torch.no_grad():
    output = model(data_train.x, data_train.edge_index)
    prob = torch.sigmoid(output[data_train.test_idx]).squeeze().cpu().numpy()
    y_pred = (prob > 0.5).astype(int)
    y_true = data_train.y[data_train.test_idx].cpu().numpy().astype(int)

# ====================== 评估指标 ======================
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)

print(f"\nTest Metrics:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")
print(f"MCC:       {mcc:.4f}")


Test Metrics:
Accuracy:  0.9790
Precision: 0.9109
Recall:    0.8695
F1-score:  0.8897
MCC:       0.8784
